In [1]:
import os
import glob
from pathlib import Path

!pip install inquirer
import inquirer

import pyspark

from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizer

import pandas as pd

%autosave 60

Autosaving every 60 seconds


In [2]:
class SparkFactory:
    def __init__(self):
        self.source_dataset_folder = Path('./dataset/')
        self.spark = SparkSession.builder.appName(__name__).getOrCreate()
    
    def validate_available_dataset(self):
        return [name for name in glob.glob(f'{self.source_dataset_folder}/**/*.csv')]

    def read_dataset(self, dataset):
        """Read source dataset with sql interface"""
        print(f'Reading dataset: {dataset}')
        try:
            self.spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(dataset) 
            data = self.spark.read.option("header", "true").csv(dataset)
        except Exception as e:
            raise e
            
        return data
    
    def _one_hot_endcoder(data):
        
        Vectorizer = CountVectorizer(inputCol="Color_Array", outputCol="Color_OneHotEncoded", vocabSize=4, minDF=1.0)
        return data
    
    def run(self):
        # validate and choose dataset to work with
        assert input('Hi! Would u like to see available datasets to process? Y/n: ') == 'Y', 'Okay, see u later!'    
        print(self.validate_available_dataset())
        dataset = input('Good! Choose one of datasets to work with pasting full path shown in previous step: ')
        
        while False:
            assert Path(dataset).is_file() == True, 'Wrong path, try one more time!'
        print(f'Nice, we gonna to preprocess {dataset}')
        data = self.read_dataset(dataset)
        # print(data.toPandas().head(10))
        
        # validate and choose method to use
        print('Available methods to apply: [OHE, B, C, D]')
        method = input('Now choose one of methods to apply ')
        if method == 'OHE':
            data = _one_hot_endcoder(data)


In [3]:
# SparkFactory().run()
data = SparkFactory().read_dataset('dataset/CICIDS2017_fine/X_train.csv')

Reading dataset: dataset/CICIDS2017_fine/X_train.csv


In [4]:
from pyspark.sql.types import StringType, DoubleType

categorical_columns = [f.name for f in data.schema.fields if isinstance(f.dataType, StringType)]
numerical_columns = [f.name for f in data.schema.fields]
print(len(categorical_columns))
print(len(numerical_columns))

123
123


In [5]:
from collections import defaultdict

data_types = defaultdict(list)

for entry in data.schema.fields:
    data_types[str(entry.dataType)].append(entry.name)
    
data_types    

defaultdict(list,
            {'StringType': ['_c0',
              'ack_psh_rst_syn_fin_cnt_0',
              'ack_psh_rst_syn_fin_cnt_1',
              'ack_psh_rst_syn_fin_cnt_2',
              'ack_psh_rst_syn_fin_cnt_3',
              'ack_psh_rst_syn_fin_cnt_4',
              'bytes_in',
              'bytes_out',
              'dst_port',
              'hdr_bin_40',
              'hdr_ccnt_0',
              'hdr_ccnt_1',
              'hdr_ccnt_2',
              'hdr_ccnt_3',
              'hdr_ccnt_4',
              'hdr_ccnt_5',
              'hdr_ccnt_6',
              'hdr_ccnt_7',
              'hdr_ccnt_8',
              'hdr_ccnt_9',
              'hdr_ccnt_10',
              'hdr_ccnt_11',
              'hdr_distinct',
              'hdr_mean',
              'intervals_ccnt_0',
              'intervals_ccnt_1',
              'intervals_ccnt_2',
              'intervals_ccnt_3',
              'intervals_ccnt_4',
              'intervals_ccnt_5',
              'intervals_cc

In [9]:
# from pyspark.ml import Pipeline
# from pyspark.ml.feature import OneHotEncoder, StringIndexer

# strings_used = [var for var in data_types["StringType"]]
# stage_string = [StringIndexer(inputCol= c, outputCol= c +"_string_encoded") for c in strings_used]
# stage_one_hot = [OneHotEncoder(inputCol= c + "_string_encoded", outputCol= c + "_one_hot") for c in strings_used]

# ppl = Pipeline(stages= stage_string + stage_one_hot)
# df = ppl.fit(data).transform(data)

In [11]:
df = pd.read_csv('dataset/CICIDS2017_fine/X_train.csv')

In [13]:
df.drop(columns='Unnamed: 0', inplace=True)

In [15]:
X = df.drop(columns='y')
y = df.y

In [24]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441116 entries, 0 to 441115
Columns: 121 entries, ack_psh_rst_syn_fin_cnt_0 to time_length
dtypes: float64(121)
memory usage: 407.2 MB
